In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from data import *
# real_dataset_path='/content/drive/MyDrive/Footprints Datasets/real.npz'
# fake_dataset_path='/content/drive/MyDrive/Footprints Datasets/fake.npz'

real_dataset_path='./data/real.npz'
fake_dataset_path='./data/fake.npz'
X_train,Y_train,_,_,X_val,Y_val=Split_Data(real_dataset_path=real_dataset_path,fake_dataset_path=fake_dataset_path)

print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_val))
print(np.shape(Y_val))

(1200, 1997, 101)
(1200, 496)
(300, 1997, 101)
(300, 496)


In [3]:
print("Minimum value",  np.min(X_train))
print("Maximum value", np.max(X_train))
X_train=preprocessing(X_train)
print("Minimum value",  np.min(X_train))
print("Maximum value", np.max(X_train))


print("Minimum value",  np.min(X_val))
print("Maximum value", np.max(X_val))
X_val=preprocessing(X_val)
print("Minimum value",  np.min(X_val))
print("Maximum value", np.max(X_val))

Minimum value 0.0
Maximum value inf
Minimum value 0.0
Maximum value 65500.0
Minimum value 0.0
Maximum value inf
Minimum value 0.0
Maximum value 65500.0


In [5]:
# Model
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.initializers import RandomUniform
import tensorflow as tf

In [6]:
# GRADED FUNCTION: model

def model(input_shape=(1997,101)):
    """
    Function creating the model's graph in Keras.

    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """

    X_input = Input(shape = input_shape)

    ### START CODE HERE ###

    # Step 1: CONV layer (≈4 lines)
    X = Conv1D(196, kernel_size=17, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)

    #   # Print output shape after Conv1D
    # print("Output shape after Conv1D:", X.shape)

    # Step 2: First GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X) # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                 # Batch normalization

    # # Step 3: Second GRU Layer (≈4 lines)
    X = GRU(units = 128, return_sequences = True)(X)   # GRU (use 128 units and return the sequences)
    X = Dropout(0.8)(X)                                 # dropout (use 0.8)
    X = BatchNormalization()(X)                                  # Batch normalization
    X = Dropout(0.8)(X)                                  # dropout (use 0.8)

    # # Step 4: Time-distributed dense layer (≈1 line)
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) # time distributed  (sigmoid)

    ### END CODE HERE ###

    model = Model(inputs = X_input, outputs = X)

    return model

In [7]:
model_eagle = model()
model_eagle.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1997, 101)]       0         
                                                                 
 conv1d (Conv1D)             (None, 496, 196)          336728    
                                                                 
 batch_normalization (Batch  (None, 496, 196)          784       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 496, 196)          0         
                                                                 
 dropout (Dropout)           (None, 496, 196)          0         
                                                                 
 gru (GRU)                   (None, 496, 128)          125184    
                                                            

In [ ]:
zeros = np.count_nonzero(Y_train == 0)
ones = np.count_nonzero(Y_train == 1)
print(f"Real Data: 0s: {zeros}, 1s:{ones}")
print(zeros/ones)

Real Data: 0s: 536097, 1s:59103
9.070554794172885


In [9]:
import keras
class_weights = {0: 1.0, 1: 10.0}  # Example class weights
@keras.saving.register_keras_serializable()
# Define weighted binary cross-entropy loss function
def weighted_binary_crossentropy(y_true, y_pred):
    # Clip predicted values to prevent log(0) and log(1) cases
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

    # Compute weighted binary cross-entropy
    weights = tf.constant([class_weights[i] for i in range(len(class_weights))])
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    return tf.reduce_mean(bce(y_true, y_pred) * weights)

In [10]:
# opt = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_eagle.compile(loss=weighted_binary_crossentropy, optimizer=opt, metrics=["accuracy"])

In [11]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)



240/240 [==============================] - 79s 295ms/step - loss: 3.4042 - accuracy: 0.7904


In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=30)

In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=10)

In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 16, epochs=10)

In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)

In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 5, epochs=1)

In [ ]:
model_eagle.fit(X_train,Y_train, batch_size = 64, epochs=1)

In [12]:
model_eagle.save('./model_test.h5')

In [14]:
loss, acc = model_eagle.evaluate(X_val, Y_val)
print(loss)
print(acc)

: 

In [ ]:
predictions = model_eagle.predict(X_val)
print(np.shape(predictions))

In [ ]:
print(predictions[0,:].T)
print(np.max(predictions[0,:]))

# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[0,:] >= 0.5).astype(int)
print(binary_labels.T)

In [ ]:
print(Y_val[0])
print(np.max(Y_val[0]))

In [ ]:
print(predictions[1].T)
print(np.max(predictions[1]))

# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[1,:] >= 0.5).astype(int)
print(binary_labels.T)

In [ ]:
print(Y_val[1])
print(np.max(Y_val[1]))

In [ ]:
loss, acc = model_eagle.evaluate(np.expand_dims(X_val[1,:,:], axis=0),np.expand_dims(Y_val[1,:], axis=0))

In [ ]:
print(predictions[50].T)
print(np.max(predictions[50]))
# Apply threshold of 0.5 to convert probabilities to binary labels
binary_labels = (predictions[50,:] >= 0.5).astype(int)
print(binary_labels.T)

In [ ]:
print(Y_val[50])
print(np.max(Y_val[50]))

In [ ]:
print(predictions[50].T)
print(np.max(predictions[50]))

print(Y_val[50])
print(np.max(Y_val[50]))

In [ ]:
np.max(predictions)

In [13]:
import keras
from keras.models import load_model
loaded_model = keras.models.load_model('./model_test.h5')

In [ ]:
loss, acc = loaded_model.evaluate(X_val, Y_val)
print(loss)
print(acc)

In [ ]:
# rand_model = model()

# opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# rand_model.compile(loss=weighted_binary_crossentropy, optimizer=opt, metrics=["accuracy"])
# loss, acc = x.evaluate(X_val, Y_val)
